### Move Cost Optimization

#### Scenario
Suppose that I have a business with three job categories of engineer, baker, and accountant. Each category has five promotion levels of 1 through 5. There are also four city locations where each of these categories of employees can work from. The company allows workers to change locations about every three years and will pay for it, but a level and position must be available. How can I use python with either scipy optimize or pytorch optimize to optimize the number of moves each year by maximizing the available moves while minimizing the total cost to staying within a certain budget?

In [1]:
# Gemini
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

# 1. Define Problem Parameters
jobs = ['engineer', 'baker', 'accountant']
levels = list(range(1, 6))
cities = ['New York', 'Los Angeles', 'Chicago', 'Houston']

num_jobs = len(jobs)
num_levels = len(levels)
num_cities = len(cities)

# For reproducibility
np.random.seed(42)

# Generate sample data
# H_jlc: Headcount limit for each (job, level, city)
headcount_limits = np.random.randint(2, 10, size=(num_jobs, num_levels, num_cities))

# E_jlc: Current number of employees
current_employees = headcount_limits - np.random.randint(0, 3, size=(num_jobs, num_levels, num_cities))

# R_jlc1c2: Number of employees requesting a move from c1 to c2
requests = np.random.randint(0, 4, size=(num_jobs, num_levels, num_cities, num_cities))
np.fill_diagonal(requests.reshape(num_jobs * num_levels, num_cities, num_cities), 0) # No moves within the same city

# C_jlc1c2: Cost of a move from c1 to c2
move_costs = np.random.randint(5000, 15000, size=(num_jobs, num_levels, num_cities, num_cities))

# Total annual budget for moves
total_budget = 750000

# 2. Map problem to MILP format (Ax <= b)

# Decision variables x are flattened into a 1D array.
# Create a map to easily find the index for a given (j, l, c1, c2) tuple.
var_map = {}
idx = 0
for j in range(num_jobs):
    for l in range(num_levels):
        for c1 in range(num_cities):
            for c2 in range(num_cities):
                if c1 == c2: continue # No moves within the same city
                var_map[(j, l, c1, c2)] = idx
                idx += 1
num_vars = len(var_map)

# Objective function: Maximize Sum(x) is equivalent to Minimize Sum(-1 * x)
c = -np.ones(num_vars)

# Bounds for each variable x: 0 <= x <= num_requests
# The upper bound is the number of requests for that specific move.
lower_bounds = np.zeros(num_vars)
upper_bounds = np.array([requests[j, l, c1, c2] for (j, l, c1, c2), i in var_map.items()])
bounds = Bounds(lower_bounds, upper_bounds)

# Integrality constraint: all variables must be integers
integrality = np.ones(num_vars)

# Build the constraint matrix A and bounds vector b
constraints = []

# Constraint 1: Budget
# Sum(cost * x) <= total_budget
budget_coeffs = np.zeros(num_vars)
for (j, l, c1, c2), i in var_map.items():
    budget_coeffs[i] = move_costs[j, l, c1, c2]
constraints.append(LinearConstraint(budget_coeffs, -np.inf, total_budget))

# Constraint 2: Headcount
# For each (j, l, c), Sum(arrivals) - Sum(departures) <= H - E
num_headcount_constraints = num_jobs * num_levels * num_cities
headcount_matrix = np.zeros((num_headcount_constraints, num_vars))
headcount_rhs = np.zeros(num_headcount_constraints) # Right-hand side (H - E)

hc_idx = 0
for j in range(num_jobs):
    for l in range(num_levels):
        for c in range(num_cities):
            # Calculate the right-hand side of the inequality
            headcount_rhs[hc_idx] = headcount_limits[j, l, c] - current_employees[j, l, c]
            
            # Populate the A matrix row for this constraint
            for (j_var, l_var, c1_var, c2_var), i_var in var_map.items():
                if j_var == j and l_var == l:
                    # Employee arrives at city c
                    if c2_var == c:
                        headcount_matrix[hc_idx, i_var] = 1
                    # Employee departs from city c
                    if c1_var == c:
                        headcount_matrix[hc_idx, i_var] = -1
            hc_idx += 1
            
constraints.append(LinearConstraint(headcount_matrix, -np.inf, headcount_rhs))

# 3. Solve the optimization problem
print("🚀 Starting optimization...")
res = milp(c=c, constraints=constraints, bounds=bounds, integrality=integrality)
print("✅ Optimization finished!")

# 4. Display the results
if res.success:
    print("\n--- Optimal Move Plan ---")
    total_moves = 0
    total_cost = 0
    approved_moves = res.x

    for (j, l, c1, c2), i in var_map.items():
        num_to_move = int(round(approved_moves[i]))
        if num_to_move > 0:
            cost = move_costs[j, l, c1, c2] * num_to_move
            total_cost += cost
            total_moves += num_to_move
            print(
                f"Move {num_to_move} {jobs[j]}(s) at Level {levels[l]} "
                f"from {cities[c1]} to {cities[c2]}"
            )

    print("\n--- Summary ---")
    print(f"Total approved moves: {total_moves}")
    print(f"Total cost: ${total_cost:,.2f}")
    print(f"Budget:     ${total_budget:,.2f}")
    print(f"Budget usage: {total_cost / total_budget:.2%}")
else:
    print("\nOptimization failed.")
    print(f"Message: {res.message}")

ValueError: All dimensions of input must be of equal length

In [2]:
# Claude
import numpy as np
from scipy.optimize import linprog
import pandas as pd
from itertools import product
import torch
import torch.optim as optim

class EmployeeRelocationOptimizer:
    def __init__(self):
        self.job_categories = ['engineer', 'baker', 'accountant']
        self.promotion_levels = [1, 2, 3, 4, 5]
        self.cities = ['city_A', 'city_B', 'city_C', 'city_D']
        
        # Create all possible position combinations
        self.positions = list(product(self.job_categories, self.promotion_levels, self.cities))
        self.n_positions = len(self.positions)
        
    def generate_sample_data(self, seed=42):
        """Generate sample data for demonstration"""
        np.random.seed(seed)
        
        # Current employee distribution (employees currently in each position)
        self.current_employees = np.random.randint(0, 20, self.n_positions)
        
        # Available positions (how many slots are available in each position)
        self.available_positions = np.random.randint(0, 25, self.n_positions)
        
        # Relocation costs (cost to move someone to each position)
        # Higher level positions and certain cities might be more expensive
        base_costs = np.random.uniform(5000, 15000, self.n_positions)
        
        # Add cost multipliers based on level and city
        for i, (job, level, city) in enumerate(self.positions):
            # Higher levels cost more
            level_multiplier = 1 + (level - 1) * 0.2
            # Some cities are more expensive
            city_multiplier = {'city_A': 1.0, 'city_B': 1.2, 'city_C': 0.8, 'city_D': 1.1}[city]
            base_costs[i] *= level_multiplier * city_multiplier
            
        self.relocation_costs = base_costs
        
        # Employee preferences/eligibility for moves (probability of wanting to move to each position)
        self.move_preferences = np.random.uniform(0.1, 0.9, (self.n_positions, self.n_positions))
        # Make diagonal elements 0 (can't move to same position)
        np.fill_diagonal(self.move_preferences, 0)
        
        return self
        
    def scipy_optimization(self, budget=500000, max_moves_per_employee=1):
        """
        Optimize using scipy linear programming
        Maximize total moves while staying within budget
        """
        print("Optimizing with SciPy Linear Programming...")
        
        # Decision variables: x[i,j] = number of employees moving from position i to position j
        n_vars = self.n_positions * self.n_positions
        
        # Objective: maximize total moves (minimize negative of total moves)
        c = -np.ones(n_vars)  # Negative because linprog minimizes
        
        # Constraints
        A_ub = []  # Inequality constraint coefficients
        b_ub = []  # Inequality constraint bounds
        A_eq = []  # Equality constraint coefficients  
        b_eq = []  # Equality constraint bounds
        
        # Budget constraint: sum of all moves * costs <= budget
        budget_constraint = np.zeros(n_vars)
        for i in range(self.n_positions):
            for j in range(self.n_positions):
                var_idx = i * self.n_positions + j
                budget_constraint[var_idx] = self.relocation_costs[j]
        A_ub.append(budget_constraint)
        b_ub.append(budget)
        
        # Capacity constraints: moves to position j <= available positions in j
        for j in range(self.n_positions):
            capacity_constraint = np.zeros(n_vars)
            for i in range(self.n_positions):
                var_idx = i * self.n_positions + j
                capacity_constraint[var_idx] = 1
            A_ub.append(capacity_constraint)
            b_ub.append(self.available_positions[j])
        
        # Supply constraints: moves from position i <= current employees in i
        for i in range(self.n_positions):
            supply_constraint = np.zeros(n_vars)
            for j in range(self.n_positions):
                var_idx = i * self.n_positions + j
                supply_constraint[var_idx] = 1
            A_ub.append(supply_constraint)
            b_ub.append(self.current_employees[i])
        
        # Bounds: all variables >= 0
        bounds = [(0, None) for _ in range(n_vars)]
        
        # Solve
        result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
        
        if result.success:
            moves_matrix = result.x.reshape(self.n_positions, self.n_positions)
            total_moves = np.sum(moves_matrix)
            total_cost = np.sum(moves_matrix * self.relocation_costs.reshape(-1, 1))
            
            print(f"SciPy Optimization Results:")
            print(f"Total moves: {total_moves:.0f}")
            print(f"Total cost: ${total_cost:,.2f}")
            print(f"Budget utilization: {total_cost/budget*100:.1f}%")
            
            return {
                'moves_matrix': moves_matrix,
                'total_moves': total_moves,
                'total_cost': total_cost,
                'success': True
            }
        else:
            print("SciPy optimization failed:", result.message)
            return {'success': False}
    
    def pytorch_optimization(self, budget=500000, learning_rate=0.01, epochs=1000):
        """
        Optimize using PyTorch with gradient descent
        This treats it as a continuous optimization problem with penalty methods
        """
        print("\nOptimizing with PyTorch...")
        
        # Initialize decision variables (moves matrix)
        moves = torch.rand(self.n_positions, self.n_positions, requires_grad=True)
        
        # Convert data to tensors
        costs = torch.tensor(self.relocation_costs, dtype=torch.float32)
        current_emp = torch.tensor(self.current_employees, dtype=torch.float32)
        available_pos = torch.tensor(self.available_positions, dtype=torch.float32)
        
        optimizer = optim.Adam([moves], lr=learning_rate)
        
        best_moves = None
        best_objective = float('-inf')
        
        for epoch in range(epochs):
            optimizer.zero_grad()
            
            # Ensure non-negative and no self-moves
            moves_pos = torch.relu(moves)
            moves_pos = moves_pos * (1 - torch.eye(self.n_positions))
            
            # Objective: maximize total moves
            total_moves = torch.sum(moves_pos)
            
            # Constraints as penalties
            # Budget constraint
            total_cost = torch.sum(moves_pos * costs.unsqueeze(0))
            budget_penalty = torch.relu(total_cost - budget) * 1000
            
            # Capacity constraints
            moves_to_positions = torch.sum(moves_pos, dim=0)
            capacity_penalty = torch.sum(torch.relu(moves_to_positions - available_pos)) * 100
            
            # Supply constraints  
            moves_from_positions = torch.sum(moves_pos, dim=1)
            supply_penalty = torch.sum(torch.relu(moves_from_positions - current_emp)) * 100
            
            # Combined objective (maximize moves, minimize penalties)
            objective = total_moves - budget_penalty - capacity_penalty - supply_penalty
            loss = -objective  # Minimize negative objective
            
            loss.backward()
            optimizer.step()
            
            if objective.item() > best_objective:
                best_objective = objective.item()
                best_moves = moves_pos.detach().clone()
            
            if epoch % 100 == 0:
                print(f"Epoch {epoch}: Moves={total_moves.item():.1f}, "
                      f"Cost=${total_cost.item():,.0f}, Objective={objective.item():.1f}")
        
        if best_moves is not None:
            final_moves = best_moves.numpy()
            total_moves = np.sum(final_moves)
            total_cost = np.sum(final_moves * self.relocation_costs.reshape(-1, 1))
            
            print(f"\nPyTorch Optimization Results:")
            print(f"Total moves: {total_moves:.0f}")
            print(f"Total cost: ${total_cost:,.2f}")
            print(f"Budget utilization: {total_cost/budget*100:.1f}%")
            
            return {
                'moves_matrix': final_moves,
                'total_moves': total_moves,
                'total_cost': total_cost,
                'success': True
            }
        else:
            return {'success': False}
    
    def analyze_results(self, result):
        """Analyze and display optimization results"""
        if not result['success']:
            print("Optimization failed!")
            return
            
        moves_matrix = result['moves_matrix']
        
        # Find top moves
        print(f"\nTop 10 Recommended Moves:")
        print("=" * 80)
        
        move_list = []
        for i in range(self.n_positions):
            for j in range(self.n_positions):
                if moves_matrix[i, j] > 0.1:  # Only show significant moves
                    from_pos = self.positions[i]
                    to_pos = self.positions[j]
                    move_list.append({
                        'from': f"{from_pos[0]} L{from_pos[1]} ({from_pos[2]})",
                        'to': f"{to_pos[0]} L{to_pos[1]} ({to_pos[2]})",
                        'employees': moves_matrix[i, j],
                        'cost_per_move': self.relocation_costs[j],
                        'total_cost': moves_matrix[i, j] * self.relocation_costs[j]
                    })
        
        # Sort by number of employees
        move_list.sort(key=lambda x: x['employees'], reverse=True)
        
        for i, move in enumerate(move_list[:10]):
            print(f"{i+1:2d}. {move['employees']:4.1f} employees: "
                  f"{move['from']} → {move['to']} "
                  f"(${move['cost_per_move']:,.0f} each, ${move['total_cost']:,.0f} total)")
    
    def compare_methods(self, budget=500000):
        """Compare both optimization methods"""
        print("Employee Relocation Optimization Comparison")
        print("=" * 60)
        
        # Generate sample data
        self.generate_sample_data()
        
        print(f"Problem Setup:")
        print(f"- Total positions: {self.n_positions}")
        print(f"- Total current employees: {np.sum(self.current_employees)}")
        print(f"- Total available positions: {np.sum(self.available_positions)}")
        print(f"- Budget: ${budget:,}")
        print(f"- Average relocation cost: ${np.mean(self.relocation_costs):,.0f}")
        
        # SciPy optimization
        scipy_result = self.scipy_optimization(budget)
        
        # PyTorch optimization
        pytorch_result = self.pytorch_optimization(budget)
        
        # Analysis
        if scipy_result['success']:
            print(f"\n" + "="*50)
            print("SCIPY RESULTS ANALYSIS:")
            self.analyze_results(scipy_result)
        
        if pytorch_result['success']:
            print(f"\n" + "="*50)  
            print("PYTORCH RESULTS ANALYSIS:")
            self.analyze_results(pytorch_result)

# Example usage
if __name__ == "__main__":
    optimizer = EmployeeRelocationOptimizer()
    
    # Run comparison with different budgets
    budgets = [300000, 500000, 750000]
    
    for budget in budgets:
        print(f"\n{'='*80}")
        print(f"OPTIMIZATION WITH BUDGET: ${budget:,}")
        print(f"{'='*80}")
        optimizer.compare_methods(budget)
        print("\n")


OPTIMIZATION WITH BUDGET: $300,000
Employee Relocation Optimization Comparison
Problem Setup:
- Total positions: 60
- Total current employees: 559
- Total available positions: 661
- Budget: $300,000
- Average relocation cost: $13,495
Optimizing with SciPy Linear Programming...
SciPy Optimization Results:
Total moves: 42
Total cost: $610,240.00
Budget utilization: 203.4%

Optimizing with PyTorch...
Epoch 0: Moves=1776.3, Cost=$24,023,612, Objective=-23723841536.0
Epoch 100: Moves=3.8, Cost=$51,023, Objective=-50.6
Epoch 200: Moves=0.4, Cost=$4,704, Objective=-7.4
Epoch 300: Moves=0.4, Cost=$4,702, Objective=-7.4
Epoch 400: Moves=0.4, Cost=$4,701, Objective=-7.4
Epoch 500: Moves=0.4, Cost=$4,699, Objective=-7.4
Epoch 600: Moves=0.4, Cost=$4,697, Objective=-7.3
Epoch 700: Moves=0.4, Cost=$4,694, Objective=-7.3
Epoch 800: Moves=0.4, Cost=$4,692, Objective=-7.3
Epoch 900: Moves=0.4, Cost=$4,689, Objective=-7.3

PyTorch Optimization Results:
Total moves: 0
Total cost: $5,037.73
Budget utili

In [3]:
# Chat GPT
import numpy as np
from scipy.optimize import linprog

# Parameters
categories = ['engineer', 'baker', 'accountant']
levels = list(range(1, 6))
cities = ['A', 'B', 'C', 'D']

n_cat, n_lvl, n_city = len(categories), len(levels), len(cities)

# Create dummy cost and availability matrices
np.random.seed(0)
costs = np.random.randint(500, 2000, size=(n_cat, n_lvl, n_city, n_city))
availability = np.random.randint(0, 2, size=(n_cat, n_lvl, n_city, n_city))

# Decision variables: x[i,j,k,l] -> flattened into 1D
num_vars = n_cat * n_lvl * n_city * n_city
c = -np.ones(num_vars)  # Objective: maximize moves => minimize -moves

# Budget constraint
flat_costs = costs.flatten()
budget = 10000
A_ub = [flat_costs]
b_ub = [budget]

# Bounds and binary availability constraint
bounds = [(0, availability[i][j][k][l]) 
          for i in range(n_cat)
          for j in range(n_lvl)
          for k in range(n_city)
          for l in range(n_city)]

# Solve
result = linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

print("Success:", result.success)
print("Total Moves:", -result.fun)
print("Total Cost:", sum(flat_costs[i] * result.x[i] for i in range(num_vars)))


Success: True
Total Moves: 16.639760837070256
Total Cost: 10000.0
